In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers


df = pd.read_csv("E:\Final year project\project\datapreprocessing\iot_combined.csv")


top_labels = {
    'PartOfAHorizontalPortScan': 1,
    'Benign': 0,
    'Okiru': 2,
    'DDoS': 3,
}

df = df[df['label'].isin(top_labels)]

balanced_df = pd.concat([df[df['label'] == label].sample(12500, replace=True) for label in top_labels])


balanced_df = balanced_df.sample(frac=1).reset_index(drop=True)
balanced_df['label'] = balanced_df['label'].map(top_labels)

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


X = balanced_df.drop(columns=['label'])
y = balanced_df['label']


num_classes = 4
y_encoded = to_categorical(y, num_classes)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Reshape the input data for LSTM
input_shape_lstm = (X_train.shape[1], 1)
X_train_reshaped_lstm = X_train.values.reshape(X_train.shape[0], *input_shape_lstm)
X_test_reshaped_lstm = X_test.values.reshape(X_test.shape[0], *input_shape_lstm)

# Define and compile the LSTM model
lstm_model = keras.Sequential([
    layers.LSTM(64, input_shape=input_shape_lstm),
    layers.Dense(num_classes, activation='softmax')
])

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model.fit(X_train_reshaped_lstm, y_train, epochs=10, validation_data=(X_test_reshaped_lstm, y_test))

# Predict using the LSTM model
X_train_features_lstm = lstm_model.predict(X_train_reshaped_lstm)
X_test_features_lstm = lstm_model.predict(X_test_reshaped_lstm)

# Define and compile the final dense layer model
dense_model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train_features_lstm.shape[1],)),
    layers.Dense(num_classes, activation='softmax')
])

dense_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the final dense layer model
dense_model.fit(X_train_features_lstm, y_train, epochs=10, validation_data=(X_test_features_lstm, y_test))

# Print accuracy for the final dense layer model
train_accuracy_dense = dense_model.evaluate(X_train_features_lstm, y_train, verbose=0)[1]
test_accuracy_dense = dense_model.evaluate(X_test_features_lstm, y_test, verbose=0)[1]
print(f"Dense Model - Train Accuracy: {train_accuracy_dense:.4f}, Test Accuracy: {test_accuracy_dense:.4f}")


Epoch 1/10
1250/1250 [==============================] - 43s 29ms/step - loss: 0.8231 - accuracy: 0.5843 - val_loss: 0.7330 - val_accuracy: 0.6432
Epoch 2/10
1250/1250 [==============================] - 33s 26ms/step - loss: 0.7678 - accuracy: 0.6102 - val_loss: 0.7271 - val_accuracy: 0.6432
Epoch 3/10
1250/1250 [==============================] - 33s 27ms/step - loss: 0.7254 - accuracy: 0.6417 - val_loss: 0.7215 - val_accuracy: 0.6433
Epoch 4/10
1250/1250 [==============================] - 34s 27ms/step - loss: 0.7233 - accuracy: 0.6419 - val_loss: 0.7268 - val_accuracy: 0.6434
Epoch 5/10
1250/1250 [==============================] - 33s 26ms/step - loss: 0.7381 - accuracy: 0.6298 - val_loss: 0.7694 - val_accuracy: 0.5958
Epoch 6/10
1250/1250 [==============================] - 33s 27ms/step - loss: 0.7346 - accuracy: 0.6303 - val_loss: 0.7210 - val_accuracy: 0.6434
Epoch 7/10
1250/1250 [==============================] - 36s 29ms/step - loss: 0.7215 - accuracy: 0.6420 - val_loss: 0.7219 -

In [3]:
from sklearn.metrics import classification_report


y_pred_lstm = np.argmax(lstm_model.predict(X_test_reshaped_lstm), axis=1)
y_true = np.argmax(y_test, axis=1)
print("LSTM Model Classification Report:")
print(classification_report(y_true, y_pred_lstm))

y_pred_dense = np.argmax(dense_model.predict(X_test_features_lstm), axis=1)

print("Dense Model Classification Report:")
print(classification_report(y_true, y_pred_dense))

313/313 [==============================] - 4s 14ms/step
LSTM Model Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.40      0.56      2442
           1       0.61      0.50      0.55      2565
           2       0.46      0.88      0.60      2513
           3       1.00      0.79      0.88      2480

    accuracy                           0.64     10000
   macro avg       0.74      0.64      0.65     10000
weighted avg       0.74      0.64      0.65     10000
313/313 [==============================] - 2s 5ms/step
Dense Model Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.40      0.56      2442
           1       0.61      0.50      0.55      2565
           2       0.46      0.88      0.60      2513
           3       1.00      0.79      0.88      2480

    accuracy                           0.64     10000
   macro avg       0.74      0.64      0.65     10000
weight